# Capstone Project Data Clean and Process Workbook
Cary Mosley, May 2020

## Import Libraries

In [134]:
import pandas as pd
import calendar, time
import datetime as dt
from datetime import date
from pandas.tseries.offsets import DateOffset
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize
import string, re 

import warnings
warnings.filterwarnings("once")

pd.reset_option('max_rows')

## Data Clean

### Clean NYT Article Data

In [35]:
#Set Years
years = range(2005,2021)

#Select keywords for articles
keywords = ['Stock','Market','Finance','Business','Price','Debt','Portfolio','SP500',
            'Nasdaq','Dow Jones']

#Create DataFrame
nyt_df_final = pd.DataFrame(columns=['Date', 'Headline','Snippet'])

#Select just the headlines and snippets containing keywords
for year in years:
    
    try:
        #read in csv data
        path = 'Data/nyt'+str(year)+'1.csv'
        nyt_df = pd.read_csv(path,index_col=None)
        nyt_df.drop(columns=['Unnamed: 0'],inplace=True)
        
        #select date and convert to datetime object
        nyt_df['Date']=nyt_df['Date'].apply(lambda x: x[0:10])
        nyt_df['Date']=pd.to_datetime(nyt_df['Date'])
        
        #all headlines are there but some snippets are missing
        nyt_df.dropna(inplace=True)
        
        #select headlines and snippets containing the keywords
        nyt_df_headlines = nyt_df[nyt_df['Headline'].str.contains('|'.join(keywords))]
        nyt_df_headlines = nyt_df_headlines[~nyt_df_headlines["Headline"].str.contains('Corrections')]
        nyt_df_snippets = nyt_df[nyt_df['Snippet'].str.contains('|'.join(keywords))]
        
        #concatenate dataframes
        nyt_df_final = pd.concat([nyt_df_final,nyt_df_headlines,nyt_df_snippets])
        print(year,'1')
    except:
        print('error',year,'1')
        continue
    
    try:
        #read in csv data
        path = 'Data/nyt'+str(year)+'2.csv'
        nyt_df = pd.read_csv(path,index_col=None)
        nyt_df.drop(columns=['Unnamed: 0'],inplace=True)
        
        #select date and convert to datetime object
        nyt_df['Date']=nyt_df['Date'].apply(lambda x: x[0:10])
        nyt_df['Date']=pd.to_datetime(nyt_df['Date'])
        
        #all headlines are there but some snippets are missing
        nyt_df.dropna(inplace=True)
        
        #select headlines and snippets containing the keywords
        nyt_df_headlines = nyt_df[nyt_df['Headline'].str.contains('|'.join(keywords))]
        nyt_df_headlines = nyt_df_headlines[~nyt_df_headlines["Headline"].str.contains('Corrections')]
        nyt_df_snippets = nyt_df[nyt_df['Snippet'].str.contains('|'.join(keywords))]
        
        #concatenate dataframes
        nyt_df_final = pd.concat([nyt_df_final,nyt_df_headlines,nyt_df_snippets])
        print(year,'2')
    except:
        print('error',year,'2')
        continue

#Set the nyt_df_final index as the Date
nyt_df_final.set_index('Date', inplace=True)
nyt_df_final.drop_duplicates(inplace=True)
#Write to pickle
nyt_df_final.to_pickle('Data/nyt_df')

error 2005 1
2006 1
error 2006 2
2007 1
2007 2
2008 1
2008 2
2009 1
2009 2
2010 1
2010 2
2011 1
2011 2
2012 1
2012 2
2013 1
2013 2
2014 1
2014 2
2015 1
2015 2
2016 1
2016 2
2017 1
2017 2
2018 1
2018 2
2019 1
2019 2
2020 1
error 2020 2


In [36]:
nyt_df_final.shape

(34930, 2)

In [37]:
nyt_df_final.head()

,Headline,Snippet
Date,,
2006-06-01,Glass Makers Fined for Price Fixing,The European Commission issued one of its larg...
2006-06-01,2 New Captains of the Economy Face Volatile Gl...,One reached the pinnacle of wealth and prestig...
2006-06-01,"For Google, M&A is Bad Business",Google on Wednesday poured cold water on specu...
2006-06-01,Nasdaq's Greifeld Plays the Tortoise in L.S.E....,"Nasdaq's chief executive, Robert Greifeld, is ..."
2006-06-01,Glass Makers Fined for Price Fixing,The European Commission issued one of its larg...


### Clean Ticker Data

In [30]:
#Import Ticker Data
spy_df = pd.read_csv('Data/spy.csv')
vix_df = pd.read_csv('Data/vix.csv')
qqq_df = pd.read_csv('Data/qqq.csv')
iwm_df = pd.read_csv('Data/iwm.csv')

In [31]:
#Convert Date to datetime
spy_df['Date'] = pd.to_datetime(spy_df['Date'])
vix_df['Date'] = pd.to_datetime(vix_df['Date'])
qqq_df['Date'] = pd.to_datetime(qqq_df['Date'])
iwm_df['Date'] = pd.to_datetime(iwm_df['Date'])

In [32]:
#Set the Date as the index
spy_df.set_index('Date', inplace=True)
vix_df.set_index('Date', inplace=True)
qqq_df.set_index('Date', inplace=True)
iwm_df.set_index('Date', inplace=True)

In [33]:
#Check for null values
print(' SPY \n',spy_df.isna().sum())
print(' VIX \n',vix_df.isna().sum())
print('QQQ \n', qqq_df.isna().sum())
print('IWM \n', iwm_df.isna().sum())


 SPY 
 Adj Close    0
dtype: int64
 VIX 
 Adj Close    0
dtype: int64
QQQ 
 Adj Close    0
dtype: int64
IWM 
 Adj Close    0
dtype: int64


There are no null values which is expected due to the data source

Now we will export all of these seperate dataframes to pickles

In [35]:

spy_df.to_pickle('Data/spy_df')
vix_df.to_pickle('Data/vix_df')
qqq_df.to_pickle('Data/qqq_df')
iwm_df.to_pickle('Data/iwm_df')

### Clean Sentiment Data

In [79]:
#Read in 3 sentiment csv files
AAII_df = pd.read_csv('Data/AAII.csv')
NAAIM_df = pd.read_csv('Data/NAAIM.csv')


In [80]:
#Convert Date to datetime format
AAII_df['Date'] = pd.to_datetime(AAII_df['Date'])

In [81]:
#Convert Date to datetime format
NAAIM_df['Date'] = pd.to_datetime(NAAIM_df['Date'])

I can see that moving the date of the NAAIM data forward by one will result in all the dates being in alignment

In [82]:
NAAIM_df['Date'] = NAAIM_df['Date'] + pd.Timedelta('1 day')

I need to adjust the first NAAIM date back two days to get them in line. After that everything is set

In [83]:

NAAIM_df['Date'].loc[0] = NAAIM_df['Date'].loc[0] - pd.Timedelta('2 days')

Checking that it worked

In [84]:
NAAIM_df['Date'].loc[0]

Timestamp('2006-06-29 00:00:00')

In [85]:
#Set the Date as the index
AAII_df.set_index('Date', inplace=True)
NAAIM_df.set_index('Date', inplace=True)


In [86]:
#Check for null values
print(' AAII \n',AAII_df.isna().sum())
print(' NAAIM \n',NAAIM_df.isna().sum())


 AAII 
 Bullish             0
Neutral             0
Bearish             0
Bull-Bear Spread    0
dtype: int64
 NAAIM 
 Mean/Average             0
Most Bearish Response    0
Quart 2 (median)         0
Most Bullish Response    0
dtype: int64


In [87]:
#Rename columns
AAII_df.rename(columns={"Bull-Bear Spread": "Spread"},inplace=True)

NAAIM_df.rename(columns={"Mean/Average": "Mean","Quart 2 (median)": "Median","Most Bearish Response": "MaxBear","Most Bullish Response": "MaxBull"},inplace=True)


In [88]:
sentiment_df = pd.concat([AAII_df,NAAIM_df],axis=1,join='outer')

In [89]:
#Show all rows
pd.set_option("max_rows", None)
sentiment_df


,Bullish,Neutral,Bearish,Spread,Mean,MaxBear,Median,MaxBull
Date,,,,,,,,
2006-06-01,0.307700,0.192300,0.500000,-0.192300,NaN,NaN,NaN,NaN
2006-06-08,0.262300,0.286900,0.450800,-0.188500,NaN,NaN,NaN,NaN
2006-06-15,0.264100,0.186100,0.549800,-0.285700,NaN,NaN,NaN,NaN
2006-06-22,0.344000,0.240000,0.416000,-0.072000,NaN,NaN,NaN,NaN
2006-06-29,0.386000,0.216400,0.397700,-0.011700,56.33,-125.0,65.00,125.0
2006-07-06,0.377000,0.196700,0.426200,-0.049200,19.44,-100.0,20.00,100.0
2006-07-13,0.365000,0.240900,0.394200,-0.029200,31.20,-50.0,25.00,150.0
2006-07-20,0.238500,0.183500,0.578000,-0.339500,18.76,-100.0,25.00,100.0
2006-07-27,0.348800,0.220900,0.430200,-0.081400,17.38,-50.0,15.00,100.0


In [91]:
#Reset show all rows option
pd.reset_option('max_rows')

I need to drop the first 4 entries of the dataframe

In [93]:
sentiment_df = sentiment_df.iloc[4:]

In [94]:
sentiment_df

,Bullish,Neutral,Bearish,Spread,Mean,MaxBear,Median,MaxBull
Date,,,,,,,,
2006-06-29,0.386000,0.216400,0.397700,-0.011700,56.33,-125.0,65.0,125.0
2006-07-06,0.377000,0.196700,0.426200,-0.049200,19.44,-100.0,20.0,100.0
2006-07-13,0.365000,0.240900,0.394200,-0.029200,31.20,-50.0,25.0,150.0
2006-07-20,0.238500,0.183500,0.578000,-0.339500,18.76,-100.0,25.0,100.0
2006-07-27,0.348800,0.220900,0.430200,-0.081400,17.38,-50.0,15.0,100.0
...,...,...,...,...,...,...,...,...
2020-04-02,0.342391,0.160326,0.497283,-0.154892,23.67,-100.0,20.5,100.0
2020-04-09,0.365994,0.187320,0.446686,-0.080692,26.74,-100.0,25.0,100.0
2020-04-16,0.348601,0.223919,0.427481,-0.078880,28.71,-100.0,35.0,100.0


In [95]:
#Exporting the sentiment dataframe to a pickle
sentiment_df.to_pickle('Data/sentiment_df')

## Pre-processing

### Sentiment and Stock Data
As the sentiment data is released weekly and there tends to be a significant amount of daily variation in stock market data I'm going to take the weekly average closing price. I'm also going to align the date index for the data

In [55]:
spy_df = pd.read_pickle('Data/spy_df')

In [56]:
spy_df=spy_df.resample('W').mean()
vix_df=spy_df.resample('W').mean()
qqq_df=spy_df.resample('W').mean()
iwm_df=spy_df.resample('W').mean()

In [57]:
spy_df

,Adj Close
Date,
2006-07-02,96.124527
2006-07-09,96.086758
2006-07-16,94.830073
2006-07-23,93.916246
2006-07-30,95.820935
...,...
2020-04-12,270.555000
2020-04-19,280.590009
2020-04-26,279.156000


In [58]:
vix_df

,Adj Close
Date,
2006-07-02,96.124527
2006-07-09,96.086758
2006-07-16,94.830073
2006-07-23,93.916246
2006-07-30,95.820935
...,...
2020-04-12,270.555000
2020-04-19,280.590009
2020-04-26,279.156000


In [59]:
qqq_df

,Adj Close
Date,
2006-07-02,96.124527
2006-07-09,96.086758
2006-07-16,94.830073
2006-07-23,93.916246
2006-07-30,95.820935
...,...
2020-04-12,270.555000
2020-04-19,280.590009
2020-04-26,279.156000


In [60]:
iwm_df

,Adj Close
Date,
2006-07-02,96.124527
2006-07-09,96.086758
2006-07-16,94.830073
2006-07-23,93.916246
2006-07-30,95.820935
...,...
2020-04-12,270.555000
2020-04-19,280.590009
2020-04-26,279.156000


Align the stock data index with the sentiment data

In [61]:
spy_df.index = spy_df.index - DateOffset(days=3)
vix_df.index = vix_df.index - DateOffset(days=3)
qqq_df.index = qqq_df.index - DateOffset(days=3)
iwm_df.index = iwm_df.index - DateOffset(days=3)

In [62]:
#Add identifier for tickers
spy_df['Ticker']= 'SPY'
vix_df['Ticker']= 'VIX'
qqq_df['Ticker']= 'QQQ'
iwm_df['Ticker']= 'IWM'

In [63]:
#Create a Combined Ticker DF and export all to pickles
ticker_df = pd.concat([spy_df,vix_df,qqq_df,iwm_df],axis=1,join='inner')

In [64]:
ticker_df

,Adj Close,Ticker,Adj Close,Ticker,Adj Close,Ticker,Adj Close,Ticker
Date,,,,,,,,
2006-06-29,96.124527,SPY,96.124527,VIX,96.124527,QQQ,96.124527,IWM
2006-07-06,96.086758,SPY,96.086758,VIX,96.086758,QQQ,96.086758,IWM
2006-07-13,94.830073,SPY,94.830073,VIX,94.830073,QQQ,94.830073,IWM
2006-07-20,93.916246,SPY,93.916246,VIX,93.916246,QQQ,93.916246,IWM
2006-07-27,95.820935,SPY,95.820935,VIX,95.820935,QQQ,95.820935,IWM
...,...,...,...,...,...,...,...,...
2020-04-09,270.555000,SPY,270.555000,VIX,270.555000,QQQ,270.555000,IWM
2020-04-16,280.590009,SPY,280.590009,VIX,280.590009,QQQ,280.590009,IWM
2020-04-23,279.156000,SPY,279.156000,VIX,279.156000,QQQ,279.156000,IWM


There are some dates missing from the stock data, for these ~20 data points I'm going to drop the values from the sentiment dataframe.

In [68]:
ticker_df=ticker_df[:-1]
spy_df=spy_df[:-1]
vix_df=vix_df[:-1]
qqq_df=qqq_df[:-1]
iwm_df=iwm_df[:-1]

In [74]:
sentiment_df = pd.read_pickle('Data/sentiment_df')

In [75]:
#Create lists of the index differences
stock_date_list=list(ticker_df.index)
sentiment_date_list=list(sentiment_df.index)
diff_list = np.setdiff1d(sentiment_date_list,stock_date_list)
diff_list

array([Timestamp('2007-03-02 00:00:00'), Timestamp('2007-07-03 00:00:00'),
       Timestamp('2007-07-06 00:00:00'), Timestamp('2008-05-02 00:00:00'),
       Timestamp('2008-08-22 00:00:00'), Timestamp('2010-03-03 00:00:00'),
       Timestamp('2010-11-12 00:00:00'), Timestamp('2012-07-06 00:00:00'),
       Timestamp('2012-11-28 00:00:00'), Timestamp('2013-12-27 00:00:00'),
       Timestamp('2014-01-03 00:00:00'), Timestamp('2014-01-22 00:00:00'),
       Timestamp('2014-03-19 00:00:00'), Timestamp('2014-11-26 00:00:00'),
       Timestamp('2015-09-29 00:00:00'), Timestamp('2016-11-16 00:00:00'),
       Timestamp('2018-10-19 00:00:00'), Timestamp('2019-07-03 00:00:00'),
       Timestamp('2019-11-29 00:00:00')], dtype=object)

In [111]:
#Drop the rows from the sentiment dataframe that dont exist in the stock data
sentiment_df.drop(diff_list,inplace=True)

In [114]:
#Check that there are no differences left
stock_date_list=list(ticker_df.index)
sentiment_date_list=list(sentiment_df.index)
diff_list = np.setdiff1d(sentiment_date_list,stock_date_list)
diff_list

array([], dtype=object)

In [115]:
ticker_df.shape

(723, 8)

In [116]:
sentiment_df.shape

(723, 8)

In [119]:
#Export to pickle
ticker_df.to_pickle('Data/ticker_df')
spy_df.to_pickle('Data/spy_df')
vix_df.to_pickle('Data/vix_df')
qqq_df.to_pickle('Data/qqq_df')
iwm_df.to_pickle('Data/iwm_df')
sentiment_df.to_pickle('Data/sentiment_df')

### New York Times Article Data
Here I will remove stop words from my NYT Data as well as tokenize and lemmatize the data before grouping into weekly time periods.

In [129]:
wn = WordNetLemmatizer()
# Create stop words
stop_words_ = set(stopwords.words('english'))

#Add filter keywords to stop words
more_stop_words = ['Stock','Market','Finance','Business','Price','Debt','Portfolio','SP500',
            'Nasdaq','Dow Jones']

stop_words_.update(more_stop_words)


In [130]:
def black_txt(token):
    '''
    This function removes tokens that are in our stop words and punctuation
    
    Parameters:
        Token - words for processing
        
    Return:
        Words without stopwords
    '''
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    '''
    This function removes unnecessary characters and lemmatizes the strings
    
    Parameters:
        Text - words for processing
        
    Return:
        Clean text
    '''
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [132]:
#read in nyt pickle
nyt_df= pd.read_pickle('Data/nyt_df')
nyt_df

,Headline,Snippet
Date,,
2006-06-01,Glass Makers Fined for Price Fixing,The European Commission issued one of its larg...
2006-06-01,2 New Captains of the Economy Face Volatile Gl...,One reached the pinnacle of wealth and prestig...
2006-06-01,"For Google, M&A is Bad Business",Google on Wednesday poured cold water on specu...
2006-06-01,Nasdaq's Greifeld Plays the Tortoise in L.S.E....,"Nasdaq's chief executive, Robert Greifeld, is ..."
2006-06-01,Glass Makers Fined for Price Fixing,The European Commission issued one of its larg...
...,...,...
2020-05-06,"What You Get for $825,000","A 1796 stone-and-clapboard home in Stockton, N..."
2020-05-06,"$825,000 Homes in New Jersey, New Mexico and W...","A 1796 stone-and-clapboard home in Stockton, a..."
2020-05-06,Plastic Shields and Homemade Lysol: How a N.Y....,The coronavirus upended the mundane routine at...


In [135]:
#Cleanning and lemmatizing the headlines and snippets
nyt_df['Headline'] = nyt_df['Headline'].apply(clean_txt)
nyt_df['Snippet'] = nyt_df['Snippet'].apply(clean_txt)

In [136]:
nyt_df

,Headline,Snippet
Date,,
2006-06-01,glass makers fin price fix,european commission issue one largest fin cart...
2006-06-01,new captain economy face volatile global market,one reach pinnacle wealth prestige dealmaker w...
2006-06-01,google bad business,google wednesday pour cold water speculation m...
2006-06-01,nasdaqs greifeld play tortoise talk,nasdaqs chief executive robert greifeld clearl...
2006-06-01,glass makers fin price fix,european commission issue one largest fin cart...
...,...,...
2020-05-06,get,stone clapboard home stockton adobe house sant...
2020-05-06,home new jersey new mexico wyoming,stone clapboard home stockton adobe house sant...
2020-05-06,plastic shield homemade lysol grocery persevere,coronavirus upend mundane routine city fresh m...


In [139]:
#there were some additional duplicates now that the text has been cleaned so dropping those
nyt_df.drop_duplicates(inplace=True)

In [140]:
nyt_df

,Headline,Snippet
Date,,
2006-06-01,glass makers fin price fix,european commission issue one largest fin cart...
2006-06-01,new captain economy face volatile global market,one reach pinnacle wealth prestige dealmaker w...
2006-06-01,google bad business,google wednesday pour cold water speculation m...
2006-06-01,nasdaqs greifeld play tortoise talk,nasdaqs chief executive robert greifeld clearl...
2006-06-01,glass makers fin price fix,european commission issue one largest fin cart...
...,...,...
2020-05-06,get,stone clapboard home stockton adobe house sant...
2020-05-06,home new jersey new mexico wyoming,stone clapboard home stockton adobe house sant...
2020-05-06,plastic shield homemade lysol grocery persevere,coronavirus upend mundane routine city fresh m...


In [141]:
nyt_df.to_pickle('Data/nyt_df')

Now that the numerical data has been cleaned and the text-based data lemmatized I will move on to some EDA in the next notebook.